In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [ ]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]==('sk-proj-'):
    print("API Key looks good so far")
else:
    print("There might be a problem with your API key?")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
class Website:
    "A utility class to represent a website we have scraped, now with links"
    url: str
    title: str
    body: str
    links: list(str)
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        if soup.body:
            for irrelevant in soup.body(['script','style','img','input']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n',strip=True)
        else:
            self.text = ''
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f'Webpage title: \n {self.title}\nWebpage Contents:\n{self.text}\n\n'


In [ ]:
link_system_prompt = 'You are provided with a list of links found on a webpage. You are about to decide which of the links would be most relevant to include in a brochure about the company, such as the links to an About page, or a Company page, or Careers/Jobs page. \n'
link_system_prompt += 'You should respond in JSON  as in this example'
link_system_prompt += """
{
    "links": {
        {"type": "about page", "url": "https://full.url/goes/here/about",}
        {"type": "careers page", "url": "https://another.full.url/careers"}
    }
}
"""

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f'Here is the list of links on the website of {website.url}'
    user_prompt += 'Please decide which of these are relevant web links for a brochure about the company, respone with the full http url like http//full.url. Do not inlude Terms and Service, Privacy, email links.\n'
    user_prompt += 'Links (some might be relevant links)'
    user_prompt += '\n'.join(website.links)
    return user_prompt

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions